# TinyDB and CHIANTI
This notebook experiments a bit with the idea of turning the CHIANTI "database" into a more proper database, i.e. one that is easily queried. We'll try out the TinyDB model here.

In [13]:
import os

import periodictable
import numpy as np
import h5py
from tinydb import TinyDB,Query,middlewares
import ChiantiPy.core as ch
import ChiantiPy.tools as ch_tools

First, create a TinyDB instance.

In [ ]:
db = TinyDB('chianti_db.json')

Read some CHIANTI information.

In [ ]:
db.insert({'element':'fe','ions':15,'type':'wgfa','data':ch_tools.io.wgfaRead('fe_15')})
db.insert({'element':'fe','ions':15,'type':'elvlc','data':ch_tools.io.elvlcRead('fe_15')})
db.insert({'element':'fe','ions':16,'type':'wgfa','data':ch_tools.io.wgfaRead('fe_16')})
db.insert({'element':'fe','ions':16,'type':'elvlc','data':ch_tools.io.elvlcRead('fe_16')})

In [ ]:
q = Query()

In [ ]:
res = db.search((q.element=='fe') & (q.ions==15) & (q.type=='wgfa'))[0]['data']

In [ ]:
db.purge()

Get some info from the CHIANTI file system.

In [2]:
class ConvertNumpyArrayMiddleWare(middlewares.Middleware):
    def __init__(self,storage_cls=TinyDB.DEFAULT_STORAGE):
        super(ConvertNumpyArrayMiddleWare,self).__init__(storage_cls)
        
    def read(self):
        data = self.storage.read()
        if type(data) is not type(None):
            for table_name in data:
                table = data[table_name]
                for element_id in table:
                    item = table[element_id]
                    if item['type']=='scups' and type(item['data']) is dict:
                        item['data']['bscups'] = np.array(item['data']['bscups'])
                        item['data']['btemp'] = np.array(item['data']['btemp'])
                    if item['type']=='psplups' and type(item['data']) is dict:
                        item['data']['cups'] = np.array(item['data']['cups'])
                        item['data']['de'] = np.array(item['data']['de'])
                        item['data']['gf'] = np.array(item['data']['gf'])
                        item['data']['splups'] = np.array(item['data']['splups'])

        return data
                    
    def write(self,data):
        for table_name in data:
            table = data[table_name]
            for element_id in table:
                item = table[element_id]
                if item['type']=='scups' and type(item['data']) is dict:
                    item['data']['bscups'] = [list(bs) for bs in item['data']['bscups']]
                    item['data']['btemp'] = [list(bt) for bt in item['data']['btemp']]
                if item['type']=='psplups' and type(item['data']) is dict:
                    item['data']['cups'] = list(item['data']['cups'])
                    item['data']['de'] = list(item['data']['de'])
                    item['data']['gf'] = list(item['data']['gf'])
                    item['data']['splups'] = [list(sp) for sp in item['data']['splups']]
        self.storage.write(data)
    
    def close(self):
        self.storage.close()

In [3]:
db = TinyDB('chianti_db.json',storage=ConvertNumpyArrayMiddleWare())

In [4]:
for el_item in os.listdir(os.environ['XUVTOP']):
    try:
        periodictable.elements.symbol(el_item.capitalize())
    except ValueError:
        continue
    for ion_item in os.listdir(os.path.join(os.environ['XUVTOP'],el_item)):
        if ion_item[-1]=='d':
            continue
        base_record = {'name':el_item.capitalize(),'ion':int(ion_item.split('_')[-1])}
        file_keys = ['wgfa','elvlc','scups','psplups']
        for fk in file_keys:
            base_record['type'] = fk
            if '.'.join([ion_item,fk]) in os.listdir(os.path.join(os.environ['XUVTOP'],el_item,ion_item)):
                base_record['data'] = True
            else:
                base_record['data'] = False
            db.insert(base_record)

In [5]:
tdQ = Query()

In [6]:
db.update({'data':ch_tools.io.scupsRead('fe_15')},(tdQ.name=='Fe')&(tdQ['type']=='scups')&(tdQ.ion==15))

[1051]

So the general idea is:

* In the Emission Model, the top level interface to the ion objects, build the CHIANTI tinyDB skeleton
* Then, for all the ions in the ion list provided to the emission model, populate the tinyDB with all relevant atomic info
* Pass database object to the ChIon object. ChIon oject will also create a base query for doing easy queries when information is needed. 
* Small info about the ion like ionization equilibrium and abundance will still be kept in memory.
* Need to modify both the emission model and the ChIon object.

What about using HDF5 instead?

In [54]:
test_ions = ['fe_9','fe_15','fe_12']
with h5py.File('tmp_chianti_db.h5','w') as hf:
    for t in test_ions:
        el = t.split('_')[0].capitalize()
        ion = t.split('_')[-1]
        #create group
        hf.create_group(os.path.join('/',el,ion))

In [117]:
with h5py.File('tmp_chianti_db.h5','a') as hf:
    for t in test_ions:
        el = t.split('_')[0].capitalize()
        ion = t.split('_')[-1]
        #elvlc group and datasets
        elvlc_grp = hf.create_group(os.path.join('/',el,ion,'elvlc'))
        _tmp = ch_tools.io.elvlcRead(t)
        for key in _tmp:
            if key=='ref':
                elvlc_grp.attrs[key] = '\n'.join(_tmp[key])
            elif type(_tmp[key]) is list or type(_tmp[key]) is type(np.array([])):
                elvlc_grp.create_dataset(key,data=_tmp[key])
            else:
                elvlc_grp.attrs[key] = _tmp[key]

In [119]:
with h5py.File('tmp_chianti_db.h5','r') as hf:
    grp = hf['/Fe/9']
    print('/Fe/9' in hf)

True


In [109]:
tmp_chianti_read = ch_tools.io.splupsRead('fe_16',filetype='psplups')

In [110]:
tmp_chianti_read.keys()

['file not found']

In [120]:
foobar = ch_tools.io.scupsRead('fe_9')

In [124]:
for k in foobar:
    print(k)
    print(np.array(foobar[k]).dtype)

btemp
object
de
float64
bscups
object
gf
float64
lvl2
int64
lvl1
int64
cups
float64
ntemp
int64
lim
float64
ttype
int64
ntrans
int64
ref
|S79
